## Praproses UPI Rev.3
#### Dataset : https://github.com/crypter7054/EDA-UPI-2022 
#### Keterangan Atribut : 
- **name** = nama dosen 
- **sinta_id**  = id dosen pada website SINTA 
- **dept** = program studi 
- **sinta_score** = skor keseluruhan SINTA 
- **sinta_3yr_score** = skor SINTA dalam 3 tahun terakhir 
- **affil_score** = skor keseluruhan afiliasi 
- **affil_3yr_score** = skor afiliasi dalam 3 tahun terakhir 
- **scopus_h_index** = indeks untuk mengukur produktivitas berdasarkan jumlah publikasi dan sitasi dosen pada Scopus 
- **gs_h_index** = indeks untuk mengukur produktivitas berdasarkan jumlah publikasi dan sitasi dosen pada Google Scholar 
- **wos_h_index** = indeks untuk mengukur produktivitas berdasarkan jumlah publikasi dan sitasi dosen pada Web of Science 
- **scopus_i10_index** = skor dosen dalam publikasi artikel yang disitasi minimal oleh 10 artikel lain pada Scopus 
- **gs_i10_index** = skor dosen dalam publikasi artikel yang disitasi minimal oleh 10 artikel lain pada Google Scholar 
- **wos_i10_index** = skor dosen dalam publikasi artikel yang disitasi minimal oleh 10 artikel lain pada Web of Science 
- **scopus_g_index** = indeks untuk mengukur produktivitas berdasarkan distribusi sitasi dari publikasi dosen pada Scopus 
- **gs_g_index** = indeks untuk mengukur produktivitas berdasarkan distribusi sitasi dari publikasi dosen pada Google Scholar 
- **wos_g_index** = indeks untuk mengukur produktivitas berdasarkan distribusi sitasi dari publikasi dosen pada Web of Science
- **scopus_article** = jumlah publikasi artikel dosen pada Scopus 
- **scopus_citation** = jumlah sitasi pada publikasi dosen pada Scopus 
- **scopus_cited_document** = jumlah publikasi lain yang disitasi oleh dosen pada Scopus 
- **gs_article** = jumlah publikasi artikel dosen pada Google Scholar 
- **gs_citation** = jumlah sitasi pada publikasi dosen pada Google Scholar 
- **gs_cited_document** = jumlah publikasi lain yang disitasi oleh dosen pada Google Scholar 
- **wos_article** = jumlah publikasi artikel dosen pada Web of Science 
- **wos_citation** = jumlah sitasi pada publikasi dosen pada Web of Science 
- **wos_cited_document** = jumlah publikasi lain yang disitasi oleh dosen pada Web of Science

### Import Library

In [1]:
%matplotlib inline
import numpy as np                  
import pandas as pd                 
import matplotlib.pyplot as plt     
import seaborn as sns
from datetime import datetime, date

sns.set_theme(style="darkgrid", palette="deep")

### Data Overview

Terdapat 2 data yang digunakan yaitu, data SINTA(terdiri dari atribut skor SINTA dan beberapa index untuk mengukur produktivitas dosen ex: Scopus H Index dsb.) dan data dosen(berisi informasi fakultas, prodi, dan NIP).

- Data dosen : df_1, df_2 ,df_3
- Data SINTA : df_4
- Data gabungan (SINTA dan dosen) : df

Data dosen dibagi menjadi 2 dikarenakan terdapat perbedaan format NIP pada data dosen, dibagi menjadi 2 yaitu df_1 berisi data dosen dengan data NIP dosen format baru dan df_2 berisi data dosen dengan data NIP dosen format lama. Kemudian ditambahkan kolom baru yaitu '*Usia*' dan '*Jenis Kelamin*' yang dihasilkan dari kolom '*NIP*'. Selanjutnya df_1 dan df_2 di-*concat* menghasilkan df_3. Terakhir df_3 digabung dengan df_4 menjadi df, yang merupakan data utama yang akan diproses.

### Praproses

#### Proses df_1

In [2]:
# read data mulai dari baris 1 sampai dengan 1133 (baris dengan data NIP format baru ex: 196202081986011002)
df_1 = pd.read_csv("C:\\Users\\LENOVO\\Scraper-SINTA-UPI-2022\\EDA data\\data_mentah\\convert\\joinSINTA.csv", 
                   sep=";", skiprows=0, nrows=1133)


# ubah tipe data kolom 'NIP' menjadi string agar dapat di-slice
df_1["NIP"]= df_1["NIP"].astype(str)


# slice data kolom 'NIP' mulai dari indeks 14 sampai dengan 3 indeks terakhir sehingga didapatkan data untuk 
# menentukan jenis kelamin (ex: 1). hasil slice data kolom NIP disimpan pada kolom 'Jenis Kelamin'.
# jika nilai data kolom 'Jenis Kelamin' = 1 maka jenis kelamin Laki-laki. sedangkan jika nilai data = 2 maka 
# jenis kelamin Perempuan
df_1["Jenis Kelamin"] = df_1["NIP"].str.slice(14, -3)
df_1["Jenis Kelamin"] = np.where(df_1["Jenis Kelamin"] == "1", "Laki-laki", "Perempuan")


# slice data kolom 'NIP' mulai dari indeks 0 sampai dengan 10 indeks terakhir sehingga didapatkan data untuk 
# menentukan usia (ex: 19620208). hasil slice data kolom NIP disimpan pada kolom 'Usia'.
# kemudian ubah format string menjadi yy/mm/dd
df_1["Usia"] = df_1["NIP"].str.slice(0, -10)
df_1['Usia'] = pd.to_datetime(df_1['Usia'], format='%Y%m%d')


# inisialiasi waktu sekarang, kemudian lakukan operasi pengurangan tahun dan bulan sekarang terhadap 
# tahun dan bulan lahir dosen
now = pd.Timestamp('now')
df_1['Usia'] = (now.year - df_1['Usia'].dt.year) - ((now.month - df_1['Usia'].dt.month) < 0)

df_1.head(1494)

,No,NIP,nama,Stat.Peg,Gol,Jabatan Fungsional,Pendidikan Terakhir,Fakultas,Kode Prodi,Nama Prodi,sinta_id,scopus_h_index,gs_h_index,sinta_3yr_score,sinta_score,affil_3yr_score,affil_score,Jenis Kelamin,Usia
0,1,196202081986011002,"Prof. Dr. H. M. Solehuddin, M.Pd., M.A.",PNS,IV/c,Guru Besar,S3,FIP,86001.0,Bimbingan Dan Konseling (S3),5995348.0,3.0,10.0,223.0,443.0,0.0,0.0,Laki-laki,60
1,2,195306031980021003,"Prof. Dr. H. Mohammad Ali, M.Pd., M.A.",PNS,IV/e,Guru Besar,S3,FIP,86021.0,Pengembangan Kurikulum (S3),5980905.0,3.0,27.0,399.0,703.0,0.0,0.0,Laki-laki,69
2,3,195404021980112001,"Prof. Dr. Hj. Ihat Hatimah, M.Pd.",PNS,IV/e,Guru Besar,S3,FIP,86005.0,Pendidikan Masyarakat (S3),5979054.0,0.0,11.0,145.0,356.0,0.0,0.0,Perempuan,68
3,4,195501011981011001,"Prof. Dr. H. Achmad Hufad, M.Ed.",PNS,IV/e,Guru Besar,S3,FIP,86002.0,Pendidikan Khusus (S3),5977826.0,5.0,0.0,976.0,1378.0,0.0,0.0,Laki-laki,67
4,5,196111091987031001,"Prof. Dr. H. Mustofa Kamil, M.Pd.",PNS,IV/e,Guru Besar,S3,FIP,86105.0,Pendidikan Masyarakat (S2),5994180.0,2.0,12.0,250.0,420.0,0.0,0.0,Laki-laki,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128,1129,198911062019032014,"Fitri Alfarisa, S.Pd., M.Pd.",PNS,III/b,Tenaga Pengajar,S2,Kampus UPI di Serang,86286.0,PGSD Kampus Serang (S1),6691682.0,2.0,4.0,93.0,138.0,0.0,0.0,Perempuan,32
1129,1130,197205121998022004,"Dr. Rina Heryani, S.Pd., M.Pd.",PNS,IV/b,Pembina Tk. I - IV/b,S3,FIP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Perempuan,50
1130,1131,196611081992031002,"Dr. Asep Mahfudz, M.Si.",PNS,IV/a,Pembina - IV/a,S3,FPIPS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laki-laki,55
1131,1132,198207082005011001,"Dr. Asep Miftahuddin, M.A.B.",PNS,III/c,Penata - III/c,S3,FPEB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laki-laki,40


#### Menampilkan jumlah dan tipe data df_1

In [3]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1133 entries, 0 to 1132
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   No                   1133 non-null   int64  
 1   NIP                  1133 non-null   object 
 2   nama                 1133 non-null   object 
 3   Stat.Peg             1133 non-null   object 
 4   Gol                  1132 non-null   object 
 5   Jabatan Fungsional   1133 non-null   object 
 6   Pendidikan Terakhir  1133 non-null   object 
 7   Fakultas             1133 non-null   object 
 8   Kode Prodi           1128 non-null   float64
 9   Nama Prodi           1129 non-null   object 
 10  sinta_id             1093 non-null   float64
 11  scopus_h_index       1093 non-null   float64
 12  gs_h_index           1093 non-null   float64
 13  sinta_3yr_score      1093 non-null   float64
 14  sinta_score          1093 non-null   float64
 15  affil_3yr_score      1093 non-null   f

#### Mengecek data pada kolom 'Kode Prodi' df_1

In [4]:
# tampilkan data kolom 'Kode Prodi' pada indeks 466
df_1['Kode Prodi'].iat[466]

nan

#### Menampilkan jumlah data kosong df_1

In [5]:
# tampilkan jumlah data kosong
df_1.isnull().sum()

No                      0
NIP                     0
nama                    0
Stat.Peg                0
Gol                     1
Jabatan Fungsional      0
Pendidikan Terakhir     0
Fakultas                0
Kode Prodi              5
Nama Prodi              4
sinta_id               40
scopus_h_index         40
gs_h_index             40
sinta_3yr_score        40
sinta_score            40
affil_3yr_score        40
affil_score            40
Jenis Kelamin           0
Usia                    0
dtype: int64

In [6]:
df_1['nama'].iloc[1009]

'HANA RUWAIDAH'

In [7]:
df_1.loc[df_1["nama"] == "HANA RUWAIDAH", "nama"] = "Hana Yunansah, M.Pd."
df_1.loc[df_1["sinta_id"] == 6675247, "sinta_id"] = 5988891

In [8]:
df_1['sinta_id'].iloc[1009]

5988891.0

In [9]:
df_1['sinta_id'].iloc[608]

6159695.0

In [10]:
df_1.at[608, 'sinta_id'] = 6778862
df_1['sinta_id'].iloc[608]

6778862.0

In [11]:
df_1['sinta_id'].iloc[804]

6682218.0

In [12]:
df_1.at[804, 'sinta_id'] = 6200015
df_1['sinta_id'].iloc[804]

6200015.0

In [13]:
df_1.head(1012)

,No,NIP,nama,Stat.Peg,Gol,Jabatan Fungsional,Pendidikan Terakhir,Fakultas,Kode Prodi,Nama Prodi,sinta_id,scopus_h_index,gs_h_index,sinta_3yr_score,sinta_score,affil_3yr_score,affil_score,Jenis Kelamin,Usia
0,1,196202081986011002,"Prof. Dr. H. M. Solehuddin, M.Pd., M.A.",PNS,IV/c,Guru Besar,S3,FIP,86001.0,Bimbingan Dan Konseling (S3),5995348.0,3.0,10.0,223.0,443.0,0.0,0.0,Laki-laki,60
1,2,195306031980021003,"Prof. Dr. H. Mohammad Ali, M.Pd., M.A.",PNS,IV/e,Guru Besar,S3,FIP,86021.0,Pengembangan Kurikulum (S3),5980905.0,3.0,27.0,399.0,703.0,0.0,0.0,Laki-laki,69
2,3,195404021980112001,"Prof. Dr. Hj. Ihat Hatimah, M.Pd.",PNS,IV/e,Guru Besar,S3,FIP,86005.0,Pendidikan Masyarakat (S3),5979054.0,0.0,11.0,145.0,356.0,0.0,0.0,Perempuan,68
3,4,195501011981011001,"Prof. Dr. H. Achmad Hufad, M.Ed.",PNS,IV/e,Guru Besar,S3,FIP,86002.0,Pendidikan Khusus (S3),5977826.0,5.0,0.0,976.0,1378.0,0.0,0.0,Laki-laki,67
4,5,196111091987031001,"Prof. Dr. H. Mustofa Kamil, M.Pd.",PNS,IV/e,Guru Besar,S3,FIP,86105.0,Pendidikan Masyarakat (S2),5994180.0,2.0,12.0,250.0,420.0,0.0,0.0,Laki-laki,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1007,1008,198211172006042001,"Novi Yanthi, S.Si., M.Pd.",PNS,III/c,Lektor,S2,Kampus UPI di Cibiru,86284.0,PGSD Kampus Cibiru (S1),6004309.0,0.0,2.0,42.0,101.0,0.0,0.0,Perempuan,39
1008,1009,198110312010122003,"Endah Silawati, S.Pd., M.Pd.",PNS,III/c,Lektor,S2,Kampus UPI di Cibiru,86280.0,PGPAUD Kampus Cibiru (S1),6004067.0,0.0,4.0,13.0,57.0,0.0,0.0,Perempuan,40
1009,1010,198301132009121005,"Hana Yunansah, M.Pd.",PNS,III/c,Lektor,S2,Kampus UPI di Cibiru,86284.0,PGSD Kampus Cibiru (S1),5988891.0,0.0,141.0,308.0,1822.0,0.0,0.0,Laki-laki,39
1010,1011,197708202005012017,"Dr. Kurniawati, S.Pd., M.Pd.",PNS,III/b,Asisten Ahli,S3,Kampus UPI di Cibiru,86284.0,PGSD Kampus Cibiru (S1),6006759.0,0.0,1.0,77.0,231.0,0.0,0.0,Perempuan,45


### Proses df_2

In [14]:
# read data mulai dari baris 1134 sampai dengan 1510 (baris dengan data NIP format lama ex: 920200519500321101)
df_2 = pd.read_csv("C:\\Users\\LENOVO\\Scraper-SINTA-UPI-2022\\EDA data\\data_mentah\\convert\\joinSINTA.csv", 
                   sep=";", skiprows=range(1, 1134), nrows=377)


# ubah tipe data kolom 'NIP' menjadi string agar dapat di-slice
df_2["NIP"]= df_2["NIP"].astype(str)


# menambahkan nilai data kolom 'NIP' yang berbeda secara manual agar digit pertama yaitu 0 dapat dibaca sebagai string 
df_2['NIP'].iat[96] = "020150819560721101"
df_2['NIP'].iat[97] = "020150819590722201"
df_2['NIP'].iat[98] = "020190319571017101"
df_2['NIP'].iat[99] = "020190319650315201"
df_2['NIP'].iat[112] = "020171119521008101"


# slice data kolom 'NIP' mulai dari indeks 15 sampai dengan 2 indeks terakhir sehingga didapatkan data untuk 
# menentukan jenis kelamin (ex: 1). hasil slice data kolom NIP disimpan pada kolom 'Jenis Kelamin'.
# jika nilai data kolom 'Jenis Kelamin' = 1 maka jenis kelamin Laki-laki. sedangkan jika nilai data = 2 maka 
# jenis kelamin Perempuan
df_2["Jenis Kelamin"] = df_2["NIP"].str.slice(15, -2)
df_2["Jenis Kelamin"] = np.where(df_2["Jenis Kelamin"] == "1", "Laki-laki", "Perempuan")


# slice data kolom 'NIP' mulai dari indeks 7 sampai dengan 3 indeks terakhir sehingga didapatkan data untuk 
# menentukan usia (ex: 19620208). hasil slice data kolom NIP disimpan pada kolom 'Usia'.
# kemudian ubah format string menjadi yy/mm/dd
df_2["Usia"] = df_2["NIP"].str.slice(7, -3)
df_2['Usia'] = pd.to_datetime(df_2['Usia'], format='%Y%m%d')


# inisialiasi waktu sekarang, kemudian lakukan operasi pengurangan tahun dan bulan sekarang terhadap 
# tahun dan bulan lahir dosen
now = pd.Timestamp('now')
df_2['Usia'] = (now.year - df_2['Usia'].dt.year) - ((now.month - df_2['Usia'].dt.month) < 0)

df_2.head(400)

,No,NIP,nama,Stat.Peg,Gol,Jabatan Fungsional,Pendidikan Terakhir,Fakultas,Kode Prodi,Nama Prodi,sinta_id,scopus_h_index,gs_h_index,sinta_3yr_score,sinta_score,affil_3yr_score,affil_score,Jenis Kelamin,Usia
0,1134,920200519500321101,"Prof. Dr. H. Sunaryo Kartadinata, M.Pd.",PT-P,NaN,Guru Besar,S3,FIP,86001.0,Bimbingan Dan Konseling (S3),5990867.0,2.0,16.0,232.0,521.0,0.0,0.0,Laki-laki,72
1,1135,920200919500802101,"Prof. Dr. Djam'an Satori, M.A.",PT-P,NaN,Guru Besar,S3,FIP,86001.0,Bimbingan Dan Konseling (S3),5991840.0,1.0,20.0,86.0,355.0,0.0,0.0,Laki-laki,72
2,1136,920171219900530201,"Mela Darmayanti, S.Pd., M.Pd.",PT,III/b,Asisten Ahli,S2,FIP,86206.0,Pendidikan Guru Sekolah Dasar (S1),6120443.0,0.0,2.0,48.0,65.0,0.0,0.0,Perempuan,32
3,1137,920171219900414201,"Nadia Aulia Nadhirah, S.Pd., M.Pd.",PT,III/b,Asisten Ahli,S2,FIP,86201.0,Bimbingan Dan Konseling (S1),6141923.0,1.0,1.0,44.0,74.0,0.0,0.0,Perempuan,32
4,1138,920190219880320201,"Tri Lestari, S.Pd., M.Pd.",PT,III/b,Asisten Ahli,S2,FIP,86201.0,Bimbingan Dan Konseling (S1),6654245.0,0.0,2.0,16.0,21.0,0.0,0.0,Perempuan,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,1506,920200819940117102,"Ishak Ariawan, S.Pd., M.Kom.",PT,III/b,Tenaga Pengajar,S2,Kampus UPI di Serang,54241.0,Sistem Informasi Kelautan (S1),6745750.0,1.0,1.0,46.0,66.0,0.0,0.0,Laki-laki,28
373,1507,920200819900404101,"La Ode Alam Minsaris, S.Pi., M.Si.",PT,III/b,Tenaga Pengajar,S2,Kampus UPI di Serang,54241.0,Sistem Informasi Kelautan (S1),6760517.0,0.0,2.0,4.0,5.0,0.0,0.0,Laki-laki,32
374,1508,920200819890313102,"Himawan Prasetiyo, S.Pi., M.Si.",PT,III/b,Tenaga Pengajar,S2,Kampus UPI di Serang,54249.0,Pendidikan Kelautan dan Perikanan Kampus Seran...,6745744.0,1.0,2.0,36.0,48.0,0.0,0.0,Laki-laki,33
375,1509,920200819900322101,"Mad Rudi, S.Pd., M.Si.",PT,III/b,Tenaga Pengajar,S2,Kampus UPI di Serang,54249.0,Pendidikan Kelautan dan Perikanan Kampus Seran...,6746212.0,0.0,1.0,4.0,5.0,0.0,0.0,Laki-laki,32


In [15]:
df_2['sinta_id'].iloc[282]

6122861.0

In [16]:
df_2.at[282, 'sinta_id'] = 6131297
df_2['sinta_id'].iloc[282]

6131297.0

In [17]:
# dftest = df_2.drop(columns=["No"], axis=1, inplace=True)
# df_2 = df_2.drop(columns=['No'])

# df_2.to_csv(r'C:\Users\LENOVO\Scraper-SINTA-UPI-2022\EDA data\data_mentah\convert\dosen_nip_lama.csv', index=False)

### Proses df_3

#### Concat df_1 dan df_2 menjadi df_3

In [18]:
# sambung df_1 dengan df_2 menjadi df_3
df_3 = pd.concat([df_1, df_2], ignore_index=True)

# drop beberapa kolom pada df_3 untuk menhindari duplikat data
df_3 = df_3.drop(columns=['scopus_h_index', 'gs_h_index', 'sinta_3yr_score', 'sinta_score', 
                        'affil_3yr_score', 'affil_score', 'No'])

df_3.head(1550)

,NIP,nama,Stat.Peg,Gol,Jabatan Fungsional,Pendidikan Terakhir,Fakultas,Kode Prodi,Nama Prodi,sinta_id,Jenis Kelamin,Usia
0,196202081986011002,"Prof. Dr. H. M. Solehuddin, M.Pd., M.A.",PNS,IV/c,Guru Besar,S3,FIP,86001.0,Bimbingan Dan Konseling (S3),5995348.0,Laki-laki,60
1,195306031980021003,"Prof. Dr. H. Mohammad Ali, M.Pd., M.A.",PNS,IV/e,Guru Besar,S3,FIP,86021.0,Pengembangan Kurikulum (S3),5980905.0,Laki-laki,69
2,195404021980112001,"Prof. Dr. Hj. Ihat Hatimah, M.Pd.",PNS,IV/e,Guru Besar,S3,FIP,86005.0,Pendidikan Masyarakat (S3),5979054.0,Perempuan,68
3,195501011981011001,"Prof. Dr. H. Achmad Hufad, M.Ed.",PNS,IV/e,Guru Besar,S3,FIP,86002.0,Pendidikan Khusus (S3),5977826.0,Laki-laki,67
4,196111091987031001,"Prof. Dr. H. Mustofa Kamil, M.Pd.",PNS,IV/e,Guru Besar,S3,FIP,86105.0,Pendidikan Masyarakat (S2),5994180.0,Laki-laki,60
...,...,...,...,...,...,...,...,...,...,...,...,...
1505,920200819940117102,"Ishak Ariawan, S.Pd., M.Kom.",PT,III/b,Tenaga Pengajar,S2,Kampus UPI di Serang,54241.0,Sistem Informasi Kelautan (S1),6745750.0,Laki-laki,28
1506,920200819900404101,"La Ode Alam Minsaris, S.Pi., M.Si.",PT,III/b,Tenaga Pengajar,S2,Kampus UPI di Serang,54241.0,Sistem Informasi Kelautan (S1),6760517.0,Laki-laki,32
1507,920200819890313102,"Himawan Prasetiyo, S.Pi., M.Si.",PT,III/b,Tenaga Pengajar,S2,Kampus UPI di Serang,54249.0,Pendidikan Kelautan dan Perikanan Kampus Seran...,6745744.0,Laki-laki,33
1508,920200819900322101,"Mad Rudi, S.Pd., M.Si.",PT,III/b,Tenaga Pengajar,S2,Kampus UPI di Serang,54249.0,Pendidikan Kelautan dan Perikanan Kampus Seran...,6746212.0,Laki-laki,32


#### Menampilkan jumlah data kosong df_3

In [19]:
# tampilkan jumlah data kosong
df_3.isnull().sum()

NIP                     0
nama                    0
Stat.Peg                0
Gol                    27
Jabatan Fungsional      0
Pendidikan Terakhir     0
Fakultas                0
Kode Prodi             21
Nama Prodi             20
sinta_id               72
Jenis Kelamin           0
Usia                    0
dtype: int64

### Proses df_4

#### Merge df_3 dan df_4 menjadi df

In [20]:
# read data SINTA sebagai df_4 kemudian drop beberapa kolom untuk menghindari duplikat
df_4 = pd.read_csv("C:\\Users\\LENOVO\\Scraper-SINTA-UPI-2022\\EDA data\\dosen.csv", sep=";", encoding='utf-8')
df_4 = df_4.drop(columns=['affil_3yr_score', 'affil_score', 'name', 'dept'])


# merge df_3 dengan df_4 menggunakan kolom 'sinta_id' menjadi df, 
# kemudian ubah tipe data kolom 'sinta_id' dan 'Kode Prodi' menjadi int64
df = (pd.merge(df_3, df_4, left_on='sinta_id', right_on='sinta_id', how='left'))
# df = df.astype({'sinta_id': 'Int64', 'Kode Prodi': 'Int64'})

df.head(1510)

,NIP,nama,Stat.Peg,Gol,Jabatan Fungsional,Pendidikan Terakhir,Fakultas,Kode Prodi,Nama Prodi,sinta_id,...,wos_g_index,scopus_article,scopus_citation,scopus_cited_document,gs_article,gs_citation,gs_cited_document,wos_article,wos_citation,wos_cited_document
0,196202081986011002,"Prof. Dr. H. M. Solehuddin, M.Pd., M.A.",PNS,IV/c,Guru Besar,S3,FIP,86001.0,Bimbingan Dan Konseling (S3),5995348.0,...,2.0,16.0,24.0,10.0,42.0,379.0,22.0,0.0,0.0,0.0
1,195306031980021003,"Prof. Dr. H. Mohammad Ali, M.Pd., M.A.",PNS,IV/e,Guru Besar,S3,FIP,86021.0,Pengembangan Kurikulum (S3),5980905.0,...,NaN,20.0,45.0,13.0,135.0,17415.0,84.0,0.0,0.0,0.0
2,195404021980112001,"Prof. Dr. Hj. Ihat Hatimah, M.Pd.",PNS,IV/e,Guru Besar,S3,FIP,86005.0,Pendidikan Masyarakat (S3),5979054.0,...,1.0,9.0,0.0,0.0,62.0,452.0,34.0,5.0,0.0,0.0
3,195501011981011001,"Prof. Dr. H. Achmad Hufad, M.Ed.",PNS,IV/e,Guru Besar,S3,FIP,86002.0,Pendidikan Khusus (S3),5977826.0,...,7.0,44.0,110.0,22.0,144.0,436.0,64.0,12.0,39.0,5.0
4,196111091987031001,"Prof. Dr. H. Mustofa Kamil, M.Pd.",PNS,IV/e,Guru Besar,S3,FIP,86105.0,Pendidikan Masyarakat (S2),5994180.0,...,1.0,9.0,8.0,3.0,71.0,1572.0,45.0,5.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1505,920200819940117102,"Ishak Ariawan, S.Pd., M.Kom.",PT,III/b,Tenaga Pengajar,S2,Kampus UPI di Serang,54241.0,Sistem Informasi Kelautan (S1),6745750.0,...,0.0,2.0,3.0,1.0,9.0,4.0,1.0,0.0,0.0,0.0
1506,920200819900404101,"La Ode Alam Minsaris, S.Pi., M.Si.",PT,III/b,Tenaga Pengajar,S2,Kampus UPI di Serang,54241.0,Sistem Informasi Kelautan (S1),6760517.0,...,0.0,0.0,0.0,0.0,4.0,6.0,2.0,0.0,0.0,0.0
1507,920200819890313102,"Himawan Prasetiyo, S.Pi., M.Si.",PT,III/b,Tenaga Pengajar,S2,Kampus UPI di Serang,54249.0,Pendidikan Kelautan dan Perikanan Kampus Seran...,6745744.0,...,1.0,1.0,5.0,1.0,7.0,8.0,3.0,0.0,0.0,0.0
1508,920200819900322101,"Mad Rudi, S.Pd., M.Si.",PT,III/b,Tenaga Pengajar,S2,Kampus UPI di Serang,54249.0,Pendidikan Kelautan dan Perikanan Kampus Seran...,6746212.0,...,NaN,0.0,0.0,0.0,5.0,7.0,1.0,0.0,0.0,0.0


#### Menampilkan jumlah data kosong df

In [21]:
# tampilkan jumlah data kosong
df.isnull().sum()

NIP                        0
nama                       0
Stat.Peg                   0
Gol                       27
Jabatan Fungsional         0
Pendidikan Terakhir        0
Fakultas                   0
Kode Prodi                21
Nama Prodi                20
sinta_id                  72
Jenis Kelamin              0
Usia                       0
sinta_score               73
sinta_3yr_score           73
scopus_h_index            73
gs_h_index                73
wos_h_index              773
scopus_i10_index          73
gs_i10_index              73
wos_i10_index            773
scopus_g_index            73
gs_g_index                73
wos_g_index              773
scopus_article            73
scopus_citation           73
scopus_cited_document     73
gs_article                73
gs_citation               73
gs_cited_document         73
wos_article               73
wos_citation              73
wos_cited_document        73
dtype: int64

#### Drop data dosen Fakultas Kedokteran (FK)

In [22]:
df = df[df["Fakultas"].str.contains("FK") == False]
df.isnull().sum()

NIP                        0
nama                       0
Stat.Peg                   0
Gol                       17
Jabatan Fungsional         0
Pendidikan Terakhir        0
Fakultas                   0
Kode Prodi                 5
Nama Prodi                 4
sinta_id                  56
Jenis Kelamin              0
Usia                       0
sinta_score               57
sinta_3yr_score           57
scopus_h_index            57
gs_h_index                57
wos_h_index              757
scopus_i10_index          57
gs_i10_index              57
wos_i10_index            757
scopus_g_index            57
gs_g_index                57
wos_g_index              757
scopus_article            57
scopus_citation           57
scopus_cited_document     57
gs_article                57
gs_citation               57
gs_cited_document         57
wos_article               57
wos_citation              57
wos_cited_document        57
dtype: int64

#### Drop data dosen yang mutasi

In [23]:
# drop data berdasarkan value dari kolom 'nama'
df = df[df['nama'] != 'Dr. Asep Mahfudz, M.Si.']
df = df[df['nama'] != 'Dr. Asep Miftahuddin, M.A.B.']

df[df['nama'] == 'Dr. Asep Miftahuddin, M.A.B.'] 

,NIP,nama,Stat.Peg,Gol,Jabatan Fungsional,Pendidikan Terakhir,Fakultas,Kode Prodi,Nama Prodi,sinta_id,...,wos_g_index,scopus_article,scopus_citation,scopus_cited_document,gs_article,gs_citation,gs_cited_document,wos_article,wos_citation,wos_cited_document


#### Menambahkan data 'Kode Prodi' dan 'Nama Prodi' dosen yang kosong

In [24]:
# tambahkan nilai pada kolom 'Kode Prodi' dan 'Nama Prodi' pada indeks tertentu
df['Kode Prodi'].iat[1129] = 86206
df['Nama Prodi'].iat[1129] = "Pendidikan Guru Sekolah Dasar (S1)"

# tambahkan nilai pada kolom 'Kode Prodi' dan 'Nama Prodi' pada indeks tertentu
df['Kode Prodi'].iat[1130] = 86284
df['Nama Prodi'].iat[1130] = "PGSD Kampus Cibiru (S1)"

# tampilkan data dengan spesific value pada kolom 'nama'
df[df['nama'] == 'Dr. Agus Mulyana, S.Pd., M.Pd.'] 

,NIP,nama,Stat.Peg,Gol,Jabatan Fungsional,Pendidikan Terakhir,Fakultas,Kode Prodi,Nama Prodi,sinta_id,...,wos_g_index,scopus_article,scopus_citation,scopus_cited_document,gs_article,gs_citation,gs_cited_document,wos_article,wos_citation,wos_cited_document
1132,197208132002121004,"Dr. Agus Mulyana, S.Pd., M.Pd.",PNS,III/b,Penata Muda Tk. I - III/b,S3,Kampus UPI di Cibiru,86284.0,PGSD Kampus Cibiru (S1),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Menambahkan data 'Kode Prodi' dosen yang kosong

In [25]:
# tambahkan nilai pada kolom 'Kode Prodi' pada indeks tertentu
df['Kode Prodi'].iat[466] = 84202

# tampilkan data dengan spesific value pada kolom 'nama'
df[df['nama'] == 'Prof. Dr. Nurjanah, M.Pd.']

,NIP,nama,Stat.Peg,Gol,Jabatan Fungsional,Pendidikan Terakhir,Fakultas,Kode Prodi,Nama Prodi,sinta_id,...,wos_g_index,scopus_article,scopus_citation,scopus_cited_document,gs_article,gs_citation,gs_cited_document,wos_article,wos_citation,wos_cited_document
466,196511161990012001,"Prof. Dr. Nurjanah, M.Pd.",PNS,IV/b,Guru Besar,S3,FPMIPA,84202.0,Pendidikan Matematika (S1),5995393.0,...,0.0,17.0,26.0,12.0,9.0,73.0,8.0,0.0,0.0,0.0


#### Drop data dosen dengan golongan kosong

In [26]:
df = df.dropna(subset=['Gol'])

#### Menampilkan jumlah data kosong df

In [27]:
# tampilkan jumlah data kosong
df.isnull().sum()

NIP                        0
nama                       0
Stat.Peg                   0
Gol                        0
Jabatan Fungsional         0
Pendidikan Terakhir        0
Fakultas                   0
Kode Prodi                 0
Nama Prodi                 0
sinta_id                  48
Jenis Kelamin              0
Usia                       0
sinta_score               49
sinta_3yr_score           49
scopus_h_index            49
gs_h_index                49
wos_h_index              742
scopus_i10_index          49
gs_i10_index              49
wos_i10_index            742
scopus_g_index            49
gs_g_index                49
wos_g_index              742
scopus_article            49
scopus_citation           49
scopus_cited_document     49
gs_article                49
gs_citation               49
gs_cited_document         49
wos_article               49
wos_citation              49
wos_cited_document        49
dtype: int64

#### Menampilkan jumlah dan tipe data akhir

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1475 entries, 0 to 1509
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   NIP                    1475 non-null   object 
 1   nama                   1475 non-null   object 
 2   Stat.Peg               1475 non-null   object 
 3   Gol                    1475 non-null   object 
 4   Jabatan Fungsional     1475 non-null   object 
 5   Pendidikan Terakhir    1475 non-null   object 
 6   Fakultas               1475 non-null   object 
 7   Kode Prodi             1475 non-null   float64
 8   Nama Prodi             1475 non-null   object 
 9   sinta_id               1427 non-null   float64
 10  Jenis Kelamin          1475 non-null   object 
 11  Usia                   1475 non-null   int64  
 12  sinta_score            1426 non-null   float64
 13  sinta_3yr_score        1426 non-null   float64
 14  scopus_h_index         1426 non-null   float64
 15  gs_h

#### Tampilkan data akhir

In [29]:
df.head(1500)

,NIP,nama,Stat.Peg,Gol,Jabatan Fungsional,Pendidikan Terakhir,Fakultas,Kode Prodi,Nama Prodi,sinta_id,...,wos_g_index,scopus_article,scopus_citation,scopus_cited_document,gs_article,gs_citation,gs_cited_document,wos_article,wos_citation,wos_cited_document
0,196202081986011002,"Prof. Dr. H. M. Solehuddin, M.Pd., M.A.",PNS,IV/c,Guru Besar,S3,FIP,86001.0,Bimbingan Dan Konseling (S3),5995348.0,...,2.0,16.0,24.0,10.0,42.0,379.0,22.0,0.0,0.0,0.0
1,195306031980021003,"Prof. Dr. H. Mohammad Ali, M.Pd., M.A.",PNS,IV/e,Guru Besar,S3,FIP,86021.0,Pengembangan Kurikulum (S3),5980905.0,...,NaN,20.0,45.0,13.0,135.0,17415.0,84.0,0.0,0.0,0.0
2,195404021980112001,"Prof. Dr. Hj. Ihat Hatimah, M.Pd.",PNS,IV/e,Guru Besar,S3,FIP,86005.0,Pendidikan Masyarakat (S3),5979054.0,...,1.0,9.0,0.0,0.0,62.0,452.0,34.0,5.0,0.0,0.0
3,195501011981011001,"Prof. Dr. H. Achmad Hufad, M.Ed.",PNS,IV/e,Guru Besar,S3,FIP,86002.0,Pendidikan Khusus (S3),5977826.0,...,7.0,44.0,110.0,22.0,144.0,436.0,64.0,12.0,39.0,5.0
4,196111091987031001,"Prof. Dr. H. Mustofa Kamil, M.Pd.",PNS,IV/e,Guru Besar,S3,FIP,86105.0,Pendidikan Masyarakat (S2),5994180.0,...,1.0,9.0,8.0,3.0,71.0,1572.0,45.0,5.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1505,920200819940117102,"Ishak Ariawan, S.Pd., M.Kom.",PT,III/b,Tenaga Pengajar,S2,Kampus UPI di Serang,54241.0,Sistem Informasi Kelautan (S1),6745750.0,...,0.0,2.0,3.0,1.0,9.0,4.0,1.0,0.0,0.0,0.0
1506,920200819900404101,"La Ode Alam Minsaris, S.Pi., M.Si.",PT,III/b,Tenaga Pengajar,S2,Kampus UPI di Serang,54241.0,Sistem Informasi Kelautan (S1),6760517.0,...,0.0,0.0,0.0,0.0,4.0,6.0,2.0,0.0,0.0,0.0
1507,920200819890313102,"Himawan Prasetiyo, S.Pi., M.Si.",PT,III/b,Tenaga Pengajar,S2,Kampus UPI di Serang,54249.0,Pendidikan Kelautan dan Perikanan Kampus Seran...,6745744.0,...,1.0,1.0,5.0,1.0,7.0,8.0,3.0,0.0,0.0,0.0
1508,920200819900322101,"Mad Rudi, S.Pd., M.Si.",PT,III/b,Tenaga Pengajar,S2,Kampus UPI di Serang,54249.0,Pendidikan Kelautan dan Perikanan Kampus Seran...,6746212.0,...,NaN,0.0,0.0,0.0,5.0,7.0,1.0,0.0,0.0,0.0


#### Export Data H Indeks Setiap Program Studi

In [30]:
# prodi_test2 = df
# df_test = prodi_test2.groupby('Nama Prodi')['Kode Prodi'].min().reset_index()
# df_test1 = prodi_test2.groupby('Nama Prodi')['Fakultas'].min().reset_index()
# df_test2 = prodi_test2.groupby('Nama Prodi')['scopus_h_index'].sum().reset_index()
# df_test3 = prodi_test2.groupby('Nama Prodi')['gs_h_index'].sum().reset_index()
# df_test4 = prodi_test2.groupby('Nama Prodi')['wos_h_index'].sum().reset_index()
# df_test5 = prodi_test2.groupby('Nama Prodi')['sinta_3yr_score'].sum().reset_index()
# df_test6 = prodi_test2.groupby('Nama Prodi')['sinta_score'].sum().reset_index()

# df_final1 = (pd.merge(df_final, df_test2, how='left'))
# df_final2 = (pd.merge(df_final1, df_test3, how='left'))
# df_final3 = (pd.merge(df_final2, df_test4, how='left'))
# df_final4 = (pd.merge(df_final3, df_test5, how='left'))
# df_final5 = (pd.merge(df_final4, df_test6, how='left'))

# df_final5 = df_final5.astype({'Kode Prodi': 'Int64', 'scopus_h_index': 'Int64', 'gs_h_index': 'Int64', 'wos_h_index': 'Int64', 'sinta_3yr_score': 'Int64', 
#                              'sinta_score': 'Int64'})

# df_final5.rename(columns = {'Nama Prodi':'Program Studi', 'scopus_h_index':'Scopus H Index', 'gs_h_index':'GS H Index',
#                            'wos_h_index':'WOS H Index', 'sinta_3yr_score':'SINTA 3yr Score', 'sinta_score':'SINTA Score'}, inplace = True)
# # df_final5.to_csv(r'C:\Users\LENOVO\Scraper-SINTA-UPI-2022\EDA data\data_mentah\convert\prodi_hindeks_ags2022.csv', index=False)
# df_final5.head()

In [31]:
# # dosen_hindeks = df.drop(columns=['nama'])
# dosen_hindeks = df[["NIP", "nama", "Jenis Kelamin", "Usia", "Stat.Peg", "Jabatan Fungsional", "Pendidikan Terakhir", "Fakultas", "Kode Prodi", "Nama Prodi"
#                    , "scopus_h_index", "gs_h_index", "wos_h_index", "sinta_id", "sinta_3yr_score", "sinta_score"]]

# dosen_hindeks = dosen_hindeks.astype({'Kode Prodi': 'Int64', 'scopus_h_index': 'Int64', 'gs_h_index': 'Int64', 'wos_h_index': 'Int64', 'sinta_3yr_score': 'Int64', 
#                              'sinta_score': 'Int64', 'sinta_id': 'Int64'})

# dosen_hindeks.rename(columns = {'Nama Prodi':'Program Studi', 'scopus_h_index':'Scopus H Index', 'gs_h_index':'GS H Index',
#                            'wos_h_index':'WOS H Index', 'sinta_3yr_score':'SINTA 3yr Score', 'sinta_score':'SINTA Score', 'nama':'Nama', 'sinta_id':'SINTA ID'}, inplace = True)

# # dosen_hindeks.to_csv(r'C:\Users\LENOVO\Scraper-SINTA-UPI-2022\EDA data\data_mentah\convert\dosen_hindeks_ags2022.csv', index=False)

# dosen_hindeks.head()

#### Export data akhir menjadi csv

In [32]:
df.to_csv(r'C:\Users\LENOVO\Scraper-SINTA-UPI-2022\EDA data\data_mentah\convert\dosen_praproses.csv', index=False)